In [ ]:
import re
import string
import scipy
import pickle
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
#from prettytable import PrettyTable
#from joblib import dump, load
from astropy.table import Table, Column
import os
import glob
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict
from sklearn.feature_selection import VarianceThreshold


In [ ]:
male_comments=" "
file_count = 0
for files in glob.glob('/content/drive/My Drive/dataEDGAD/Male/*.txt'):
  if file_count < 70:    # balanced
    file1 = open(files,encoding="utf8")
    male_comments += "Id : " + os.path.splitext(os.path.basename(files))[0] + "\n"
    male_comments += file1.read()
    file_count += 1


female_comments=" "
file_count = 0
for files in glob.glob('/content/drive/My Drive/dataEDGAD/Female/*.txt'):
  if file_count < 70:     # balanced
    file1 = open(files,encoding="utf8") 
    female_comments += "Id : " + os.path.splitext(os.path.basename(files))[0] + "\n" 
    female_comments += file1.read()
    file_count += 1

combine_data=male_comments+female_comments



In [ ]:
#Remove URLs
#female_comments = re.sub(r"http\S+", "", female_comments)
#male_comments = re.sub(r"http\S+", "", male_comments)
#split
male_comments=male_comments.split("\n")
female_comments=female_comments.split("\n")

male_comments_tweet = ""
female_comments_tweet = ""
male_id = ""
female_id = ""
male_ids = ""
female_ids = ""
tweet_count = 0
# remove data and lang from text
for line in male_comments :
  if line.startswith("Id : "):
    #Get id:
    line = re.sub("Id : ", "",line)
    male_id = line
    tweet_count = 0
  if line.startswith("Tweet: ") and tweet_count < 80: # each user number of tweets
    #Remove Tweet:
    line = re.sub("Tweet: ", "",line)
    male_comments_tweet += line + "\n"
    male_ids += male_id + "\n"
    tweet_count += 1
  

for line in female_comments:
  if line.startswith("Id : "):
    #Get id:
    line = re.sub("Id : ", "",line)
    female_id = line 
    tweet_count = 0
  if line.startswith("Tweet: ")and tweet_count < 80: # each user number of tweets
    #Remove Tweet: 
    line = re.sub("Tweet: ", "",line)
    female_comments_tweet += line + "\n"
    female_ids += female_id + "\n"
    tweet_count += 1
  

male_comments=male_comments_tweet.split("\n")
female_comments=female_comments_tweet.split("\n")
male_ids_user=male_ids.split("\n")
female_ids_user=female_ids.split("\n")




In [ ]:
#TODO: YOUR CODE GOES HERE
#comments=comments.replace('@', '[AT]', 1).replace('.', '').replace('[AT]', '#')

male_comments=pd.DataFrame(male_comments)
female_comments=pd.DataFrame(female_comments)
male_comments['gender']='male'
male_comments['idUser']=male_ids_user
female_comments['gender']='female'
female_comments['idUser']=female_ids_user
# male_comments = male_comments.sample(frac=1)
print(male_comments.head(10))
print(female_comments.head(10))

# male_comments_data=pd.DataFrame(male_comments_data)
# male_comments_data['gender']='male'
# # print(male_comments_datas)
# print(male_comments_data.head(10))

                                                   0 gender idUser
0  RT @MoSalah: كل عام وأنتم بخير ورمضان كريم ربن...   male       
1  RT @nourine_hammad: This drawing is a tribute ...   male      8
2                         ✋🤚 https://t.co/Rj2W1Z4gQE   male      8
3                          😂 https://t.co/ggkbGY5jAT   male      8
4  RT @MoSalah: It was a very tough night, but I'...   male      8
5                                               1h45   male      8
6                                               24h.   male      8
7                                       RT @LFC: 👏👏👏   male      8
8            RT @MoSalah: 🧐🧐 https://t.co/REESq5bg2t   male      8
9  RT @DanielAgger: Thanks for the visit boys😎hop...   male      8
                                                   0  gender idUser
0                           ازاي ؟؟ بجد ... ازاي ؟!!  female     95
1           @ahmedkadry92 @Ahmad_elAtriby بس يا اخ 😒  female     95
2                                   @Ahmad_elAtriby   femal

In [ ]:
#Combine Male and female tweets
tweets = [male_comments, female_comments]
tweets = pd.concat(tweets)
tweets.index = range(len(tweets))
print(len(tweets))
tweets.rename(columns={0:'tweets'}, inplace=True)
print(tweets.head(5)['tweets'])

10908
0    RT @MoSalah: كل عام وأنتم بخير ورمضان كريم ربن...
1    RT @nourine_hammad: This drawing is a tribute ...
2                           ✋🤚 https://t.co/Rj2W1Z4gQE
3                            😂 https://t.co/ggkbGY5jAT
4    RT @MoSalah: It was a very tough night, but I'...
Name: tweets, dtype: object


In [ ]:
#Trian Test Split
x_train, x_test, y_train, y_test = train_test_split(tweets['tweets'], tweets['gender'], test_size=0.33, random_state=7)

In [ ]:
print("""train Dataset:
=================================================================""")
train_dataset=pd.DataFrame(x_train)
train_dataset['gender']=pd.DataFrame(y_train)
print(train_dataset.head(10))

train Dataset:
                                                 tweets  gender
7262         RT @maielmansy: 💞💞 https://t.co/13rq6SDSix  female
5723        RT @SummerNazif: نفسي أصحى ألاقي الصيف خلِص  female
4856  RT @tarektal3at: مصدر لـ@Yallakoranow : علي جب...    male
7092        RT @HebaMohammed112: اللهم اعد ترتيب المشهد  female
7370  @Legenagy TO KEEP ME FROM GETTING TO YOOOU BAA...  female
8322  RT @mhmdshehata22: عدلى القيعى لن ينسى التاريخ...  female
1808                            https://t.co/g8ASEtwIB9    male
5525  أوكيه خليت زميلي في الشغل يعمل ڤوت لصلاح بالعا...  female
1441  مكملين مشوار اعظم نادي في الكون 🔴🦅 #اهلاوي htt...    male
2416                  @maryemahmeed هيبقى وحش من غيرك 😢    male


In [ ]:
print("""test Dataset:
================================================================""")
test_dataset=pd.DataFrame(x_test)
test_dataset['gender']=pd.DataFrame(y_test)
print(test_dataset.head(10))

test Dataset:
                                                  tweets  gender
9392   i just had one of those dramatic fights with m...  female
6512   RT @doryhassan: أخاف أن ينتهي العالم ولم نحقق ...  female
10578  RT @MuhammadEnaba: "بلاش تخلي قلبك أبيض لأن ال...  female
9282   RT @3HDALTMEMY: ستبقي ابتسامة #عهد_التميمي هي ...  female
7226   RT @KarlreMarks: So an economist walks into a ...  female
5216   RT @OmaarCupo: Girls on Instagram stories http...    male
10184                      @k_tarekk tab 3ayzeen men dah  female
7030                 قلباااااااي https://t.co/800taBqlY8  female
509          No has fewer letters than kosdeenomelygabk.    male
10390  RT @nadinkhaled_: ممكن استحمل اي حاجه في الدني...  female


In [ ]:
print(""" 'Male' in train data:
============================================================""")
male_train_dataset=train_dataset[train_dataset.gender=='male']
print(male_train_dataset.head(10))

print("""Words used by 'feMale' in train data:
============================================================""")
female_train_dataset=train_dataset[train_dataset.gender=='female']
print(female_train_dataset.head(10))

 'Male' in train data:
                                                 tweets gender
4856  RT @tarektal3at: مصدر لـ@Yallakoranow : علي جب...   male
1808                            https://t.co/g8ASEtwIB9   male
1441  مكملين مشوار اعظم نادي في الكون 🔴🦅 #اهلاوي htt...   male
2416                  @maryemahmeed هيبقى وحش من غيرك 😢   male
108   "ماذا سيحدث لو حرّكت شيئًا من كبريائك وطمأنت خ...   male
3907                                                  .   male
743                            @YourFavMemee مش محتاجها   male
4470      متوقع بداية كأس العالم خلال ١٠ أيام من دلوقتي   male
391   @trablla ماالراجل ممكن يديني حاجه تانيه علي ان...   male
3480                  حد يقول لصلاح يقفل الموبايل وينام   male
Words used by 'feMale' in train data:
                                                 tweets  gender
7262         RT @maielmansy: 💞💞 https://t.co/13rq6SDSix  female
5723        RT @SummerNazif: نفسي أصحى ألاقي الصيف خلِص  female
7092        RT @HebaMohammed112: اللهم اعد ترتيب المشه

In [ ]:
dict_words_male=defaultdict(int)
count=0
for x in male_train_dataset['tweets']:
    words=x.split(" ")
    for y in words:
        dict_words_male[y]=dict_words_male[y]+1

print(" MALE. :  " , dict_words_male)

dict_words_female=defaultdict(int)
count=0
for x in female_train_dataset['tweets']:
    words=x.split(" ")
    for y in words:
        dict_words_female[y]=dict_words_female[y]+1

print(" FEMALE. :  " , dict_words_female)

 MALE. :   defaultdict(<class 'int'>, {'RT': 746, '@tarektal3at:': 1, 'مصدر': 2, 'لـ@Yallakoranow': 1, ':': 18, 'علي': 79, 'جبر': 1, 'لا': 110, 'يعلم': 4, 'بانتقاله': 1, 'لصفوف': 1, 'الأسيوطي': 1, 'واللاعب': 1, 'يُريد': 1, 'البقاء': 6, 'في': 501, 'الزمالك.': 1, 'https://t.co/g8ASEtwIB9': 1, 'مكملين': 1, 'مشوار': 3, 'اعظم': 1, 'نادي': 6, 'الكون': 3, '🔴🦅': 1, '#اهلاوي': 1, 'https://t.co/MS13HRi4h3': 1, '@maryemahmeed': 2, 'هيبقى': 6, 'وحش': 7, 'من': 406, 'غيرك': 8, '😢': 1, '"ماذا': 1, 'سيحدث': 1, 'لو': 116, 'حرّكت': 1, 'شيئًا': 2, 'كبريائك': 1, 'وطمأنت': 1, 'خوفي': 1, 'عليك؟': 1, 'ماذا': 15, 'يحدث': 1, 'أخبرتني': 1, 'أنك': 4, 'بخير': 8, 'وهربتَ': 1, 'بعيدًا': 1, 'عنّي': 1, 'للمرة': 1, 'الأل…': 1, 'https://t.co/Z9R9V3lFNN': 1, '.': 40, '@YourFavMemee': 8, 'مش': 243, 'محتاجها': 1, 'متوقع': 3, 'بداية': 3, 'كأس': 14, 'العالم': 40, 'خلال': 6, '١٠': 5, 'أيام': 5, 'دلوقتي': 19, '@trablla': 1, 'ماالراجل': 1, 'ممكن': 49, 'يديني': 2, 'حاجه': 32, 'تانيه': 3, 'انها': 13, 'عويسي😂': 1, 'حد': 105, 'يقو

In [ ]:
# fit method
def text_fit(X, y, model,ml_model):
    
    X_c = model.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_c, y, random_state=0)
    ml = ml_model.fit(X_train, y_train)
    
    y_pred = ml_model.predict(X_test)
    print("accuracy score of ",accuracy_score(y_test, y_pred))
    # auc=roc_auc_score(y_test, ml_model.predict_proba(X_test)[:,1])
    # print("AUC score",auc)
    # print('')
    # print(classification_report(y_test, y_pred, target_names=["negative", "positive"]))

In [ ]:

#RandomForest model on TFIDF Uni-gram
tfidf = TfidfVectorizer()
text_fit(tweets['tweets'],tweets['gender'], tfidf, RandomForestClassifier())


accuracy score of  0.6784011734506784


In [ ]:
#RandomForest model on TFIDF + ngram 1-2
# tfidf_n = TfidfVectorizer(ngram_range=(1,2))
# text_fit(tweets['tweets'],tweets['gender'], tfidf_n, RandomForestClassifier())

In [ ]:
#RandomForest model on TFIDF + ngram 1-3
tfidf_nc = TfidfVectorizer(ngram_range=(1,3))
text_fit(tweets['tweets'],tweets['gender'], tfidf_nc, RandomForestClassifier())

accuracy score of  0.6776677667766776


In [ ]:
# RandomForest with Uni-gram BoW and stop_words
cv = CountVectorizer()   
text_fit(tweets['tweets'],tweets['gender'], cv, RandomForestClassifier())

accuracy score of  0.6817015034836817


In [ ]:
# RandomForest on CV + ngram 1-2
# cv_n = CountVectorizer(ngram_range=(1,2))   
# text_fit(tweets['tweets'],tweets['gender'], cv_n, RandomForestClassifier())

In [ ]:
# RandomForest on CV + ngram 1-3
cv_nc = CountVectorizer(ngram_range=(1,3))   
text_fit(tweets['tweets'],tweets['gender'], cv_nc, RandomForestClassifier())

accuracy score of  0.6703337000366704


In [ ]:

# Logistic regression model on TFIDF
tfidf = TfidfVectorizer()
text_fit(tweets['tweets'],tweets['gender'], tfidf, LogisticRegression(penalty='l2'))

accuracy score of  0.6901356802346902


In [ ]:
# Logistic regression model on TFIDF
tfidf = TfidfVectorizer(ngram_range=(1,3))
text_fit(tweets['tweets'],tweets['gender'], tfidf, LogisticRegression(penalty='l2'))

accuracy score of  0.6941694169416942


In [ ]:
# Logistic regression model on TFIDF
cv = CountVectorizer()
text_fit(tweets['tweets'],tweets['gender'], cv, LogisticRegression(penalty='l2'))

accuracy score of  0.6989365603226989


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
# Logistic regression model on TFIDF
cv = CountVectorizer(ngram_range=(1,3))
text_fit(tweets['tweets'],tweets['gender'], cv, LogisticRegression(penalty='l2'))

accuracy score of  0.6901356802346902
